<a href="https://colab.research.google.com/github/moseskim/bert_nlp/blob/main/section_4/01_simple_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 파인 튜닝

최소한의 코드로 파인 튜닝을 구현합니다.  
사전 학습 완료 모델을 몇 차례 추가로 훈련합니다.

## 라이브러리 설치

라이브러리인 Transformers를 설치합니다.  
https://huggingface.co/transformers/index.html

In [ ]:
!pip install transformers

## 모델 로딩

`BertForSequenceClassification`을 사용해 사전 학습 완료 모델을 로딩합니다.

In [ ]:
from transformers import BertForSequenceClassification

sc_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", return_dict=True)
print(sc_model.state_dict().keys())

## 최적화 알고리즘

여기에서는 최적화 알고리즘으로 `AdamW`를 채용합니다.  
`AdamW`는 오리지널 `Adam`의 가중치 감쇠에 관한 식을 변경한 것입니다.  
https://arxiv.org/abs/1711.05101

In [ ]:
from transformers import AdamW

optimizer = AdamW(sc_model.parameters(), lr=1e-5)

## Tokenizer 설정

`BertTokenizer`를 사용해 문장을 단어로 분할하고 `id`로 변환합니다.  
`BertForSequenceClassification` 모델 훈련 시에는 입력 외에 어텐션 mask를 전달해야 합니다. BertTokenizer`를 사용해 이것을 얻을 수 있습니다.

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sentences = ["I love baseball.", "I hate baseball."]
tokenized = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
print(tokenized)

x = tokenized["input_ids"]
attention_mask = tokenized["attention_mask"]

## 파인 튜닝

사전에 학습 완료한 모델을 추가로 훈련시킵니다.

In [ ]:
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt

sc_model.train()
t = torch.tensor([1,0])  # 문장 분류

weight_record = []  # 가중치 기록

for i in range(100):
    y = sc_model(x, attention_mask=attention_mask)
    loss = F.cross_entropy(y.logits, t)
    loss.backward()
    optimizer.step()

    weight = sc_model.state_dict()["bert.encoder.layer.11.output.dense.weight"][0][0].item()
    weight_record.append(weight)

plt.plot(range(len(weight_record)), weight_record)
plt.show()

추가 훈련에 따라 가중치가 조정된 상태를 확인할 수 있습니다.